# What I am Going to DO to get it working
- take the literally just 6_1 and see what is wrong with the colorlist

In [3]:
import numpy as np 
import pandas as pd 
import csv
from itertools import *
from fractions import Fraction

In [37]:
def ColourList(matrix,p):
    #get number of rows as a variable and initialize lists for later. 
    number_rows = len(matrix)

    possible_colours = []
    for i in range(p):
        possible_colours.append(i)
    
    colourlists = []
    colours = []
    pivots = []
    free_variables = []


    #Loop through each of the rows
    for i in range(number_rows):
        #set a flag for when a pivot is in the row. 
        pivot_found = False
        
        #Loop through each of the columns within the row
        for j in range( len( matrix[i] ) ):
            #If we haven't found a pivot yet, keep going further right
            if pivot_found == False:
                #If the value of this entry isn't 0, it's the pivot!
                if matrix[i][j] > 0:
                    #Add the location of the entry to our list pivots. 
                    pivot_location = j
                    pivots.append(pivot_location)
                    pivot_found = True
                    
                    
    #Go through each of the variables (rows). 
    #If the variable doesn't have a pivot, it is a free variable. Add to the list of free variables.
    for i in range(number_rows):
        if i not in pivots:
            free_variables.append(i)
    
    # All possible assignments of colours to the free variables
    free = list(product(possible_colours, repeat = len(free_variables)))

    free_colours=[]

    for colours in free:
        colours = list(colours)
        free_colours.append(colours)

    # Get rid of rotations of each colour list
    for colourlist in free_colours:
        colourlist = list(colourlist)
        for i in range(p):
            new_list = []
            for colour in colourlist:
                new_colour=(colour+i+1) %p
                new_list.append(new_colour)

            if new_list != colourlist and new_list in free_colours:
                free_colours.remove(new_list)

    # Get rid of reflections of each colour list
    for colourlist in free_colours:
        colourlist = list(colourlist)
        new_list = []
        for colour in colourlist:
            new_colour = (p-colour) %p
            new_list.append(new_colour)
        if new_list != colourlist and new_list in free_colours:
            free_colours.remove(new_list)

    trivial_colouring = []
    for i in range(len(free_variables)):
        trivial_colouring.append(0)

    free_colours.remove(trivial_colouring)

    for permutation in free_colours:

        colour_dict={}

        colours = []

        for i in range(len(permutation)):
            colour = list(permutation)[i]
            colour_dict[free_variables[i]] = colour

        for i in range(len(pivots)):
                colour = 0
                row = matrix[i]
                for column in range(len(row)):
                    if column not in pivots:
                        colour -= row[column]*colour_dict[column]
                colour = colour%p
                colour_dict[pivots[i]] = colour
     
        for i in range(len(matrix)):
                colours.append(colour_dict[i])

        colourlists.append(colours)


    for colourlist in colourlists:
        for i in range(len(colourlist)):
            if colourlist[i] ==0:
                colourlist[i] = p  
        
    return colourlists

In [7]:
fields = [ 'Name', 'Row Reduced Coloring Matrix']

newfields = [ 'Name', 'Color List']

filename = "./data/name_reducedmatrixfromgauss_3colorable.csv"

line_count = 0

rawfile = open(filename, 'r')

reader = csv.reader(rawfile)

new_LoL = []

ONLY_UNTIL = 50

while line_count<50:
    for row in reader:
        if line_count == 0:
            new_LoL.append(newfields)
            line_count = line_count + 1
        else:
            strings = row[1][2:-2].split("], [")
    #        print(strings)
            list_of_lists = [ list(crossing.split(",")) for crossing in strings ]
    #        print(list_of_lists)
            list_of_intlists = [ [int(n) for n in string] for string in list_of_lists ]
            list_of_colorlists = ColourList( list_of_intlists )
            if len(list_of_colorlists[0])%2 == 0:
                new_LoL.append([row[0],list_of_colorlists])
                line_count = line_count + 1
            else:
                for i in range(len(list_of_colorlists)):
                    zeroth_color = list_of_colorlists[i][0]
                    list_of_colorlists[i].append(zeroth_color)
                new_LoL.append([row[0],list_of_colorlists])
                line_count = line_count + 1


rawfile.close()

In [8]:
colors_only = [row[1] for row in new_LoL][1:]

In [9]:
#colors_only

In [10]:
new_clist = []
new_colorlists = []
new_colors_only = []
for colorlists in colors_only:
    #print(colorlists)
    for clist in colorlists:
        #print(clist)
        new_clist = [((i+1)%3)+1 for i in clist]
        new_colorlists.append(new_clist)
        new_clist = []
    new_colors_only.append(new_colorlists)
    new_colorlists = []

#new_colors_only
        

In [28]:
df = pd.read_csv("./data/name_reducedmatrixfrombraid_3colorable.csv")#. index_col = "name")

In [35]:
rrcm = []

for matrix in df["Row Reduced Coloring Matrix"].values.tolist():
    strings = matrix[2:-2].split("], [")
#        print(strings)
    list_of_lists = [ list(crossing.split(",")) for crossing in strings ]
#        print(list_of_lists)
    list_of_intlists = [ [int(n) for n in string] for string in list_of_lists ]
    rrcm.append(list_of_intlists)
    list_of_intlists = []
    list_of_lists = []
    strings = []
    #list_of_colorlists = ColourList( list_of_intlists ,3 )


In [47]:
pd.to_csv("pdcolorlists.csv",pd.DataFrame(colorlists))

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,"[2, 3, 1]",None,None,None,None,None,None,None,None,None,None,None,None
1,"[2, 1, 3, 2, 1, 3, 1]",None,None,None,None,None,None,None,None,None,None,None,None
2,"[2, 1, 3, 3, 2, 1, 3, 3, 1]",None,None,None,None,None,None,None,None,None,None,None,None
3,"[2, 3, 3, 2, 1, 3, 1]",None,None,None,None,None,None,None,None,None,None,None,None
4,"[2, 3, 3, 1, 2, 3, 3, 1]",None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1079,"[2, 1, 3, 3, 1, 2, 3, 2, 3, 1, 3, 1]",None,None,None,None,None,None,None,None,None,None,None,None
1080,"[2, 3, 3, 2, 2, 1, 2, 3, 1, 3, 3, 3, 1]","[2, 3, 1, 3, 1, 2, 2, 1, 1, 2, 3, 1, 3]","[1, 3, 1, 2, 3, 3, 1, 1, 2, 2, 3, 1, 1]","[3, 3, 1, 1, 2, 1, 3, 1, 3, 2, 3, 1, 2]",None,None,None,None,None,None,None,None,None
1081,"[1, 1, 1, 2, 2, 1, 1, 3, 3, 2, 2, 2, 3, 1]",None,None,None,None,None,None,None,None,None,None,None,None
1082,"[1, 1, 1, 2, 2, 1, 1, 3, 2, 2, 2, 3, 3, 1]",None,None,None,None,None,None,None,None,None,None,None,None


In [46]:
count = 0

for x in enumerate(colorlists):
    for y in x[1]:
        count += 1
    if count > 4:
        print(count)
        print(x)
    count = 0


13
(496, [[2, 3, 3, 2, 1, 2, 1, 3, 3, 3, 1, 2, 3, 3, 3, 3, 1], [3, 2, 1, 2, 1, 3, 1, 3, 1, 2, 1, 2, 3, 2, 3, 1, 3], [2, 2, 1, 1, 2, 2, 2, 3, 1, 2, 2, 1, 3, 2, 3, 1, 1], [1, 2, 1, 3, 3, 1, 3, 3, 1, 2, 3, 3, 3, 2, 3, 1, 2], [2, 2, 3, 3, 3, 1, 3, 3, 1, 2, 2, 3, 3, 2, 1, 3, 3], [1, 2, 3, 2, 1, 3, 1, 3, 1, 2, 3, 2, 3, 2, 1, 3, 1], [3, 2, 3, 1, 2, 2, 2, 3, 1, 2, 1, 1, 3, 2, 1, 3, 2], [2, 1, 1, 2, 1, 1, 1, 3, 2, 1, 3, 2, 3, 1, 1, 1, 3], [1, 1, 1, 1, 2, 3, 2, 3, 2, 1, 1, 1, 3, 1, 1, 1, 1], [3, 1, 1, 3, 3, 2, 3, 3, 2, 1, 2, 3, 3, 1, 1, 1, 2], [2, 3, 2, 1, 2, 1, 2, 3, 3, 3, 1, 1, 3, 3, 1, 2, 3], [1, 3, 2, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 1, 2, 1], [3, 3, 2, 2, 1, 2, 1, 3, 3, 3, 3, 2, 3, 3, 1, 2, 2]])
13
(570, [[3, 1, 2, 1, 2, 3, 1, 2, 2, 3, 1, 3, 3, 3, 2, 3, 3, 3, 1], [2, 3, 1, 1, 2, 1, 3, 1, 1, 1, 1, 3, 2, 3, 2, 1, 3, 1, 3], [2, 1, 3, 2, 1, 1, 1, 3, 3, 1, 2, 3, 2, 3, 1, 1, 3, 1, 1], [2, 2, 2, 3, 3, 1, 2, 2, 2, 1, 3, 3, 2, 3, 3, 1, 3, 1, 2], [2, 3, 1, 3, 3, 3, 3, 1, 1, 2, 3, 3, 1, 3, 3, 3, 1, 3,

In [10]:
df = pd.read_csv("./data/knotinfo_data_complete.csv",index_col=False)#. index_col = "name")

In [3]:
max(df["determinant"])

377

In [15]:
all_primes = [3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71,73,79,83,89,97,101,103,107,109,113,127,131,137,139,149,151,157,163,167,173,179,181,191,193,197,199,211,223,227,229,233,239,241,251,257,263,269,271,277,281,283,293,307,311,313,317,331,337,347,349,353,359,367,373]

In [16]:
#len(df[df["determinant"]%373==0])

In [17]:
primes_used = []
for p in all_primes:
    p_colorable_knots = df[df["determinant"]%p==0]
    if len(p_colorable_knots) != 0:
        primes_used.append(p)
#         p_colorable_gauss = p_colorable_knots[["name","gauss_notation"]]
#         p_colorable_gauss.to_csv("./data/name_gaussfromgauss_"+str(p)+"colorable.csv",index=False)
#         p_colorable_gauss = []
    else:
        continue
    p_colorable_knots=[]

In [19]:
print(primes_used)

[3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 313, 353]


In [9]:
df.head(10)

,name,category,knot_atlas,alternating,name_rank,dt_name,dt_rank,dt_notation,classical_conway_name,conway_notation,...,strongly_quasipositive,quasipositive,positive_braid_notation,positive_pd_notation,strongly_quasipositive_braid_notation,quasipositive_braid_notation,fd_clasp_number,width,torsion_numbers,td_clasp_number
1,4_1,4,4.1,Y,3,4a_1,3,"[4, 6, 8, 2]",4_1,[22],...,N,N,does not exist,does not exist,does not exist,does not exist,1.0,8.0,"{{2,{5}},{3,{4,4}},{4,{3,15}},{5,{11,11}},{6,{...",1
2,5_1,5,5.1,Y,4,5a_2,5,"[6, 8, 10, 2, 4]",5_1,[5],...,Y,Y,"{1,1,1,1,1}","{{2,8,3,7},{4,10,5,9},{6,2,7,1},{8,4,9,3},{10,...","{1,1,1,1,1}","{1,1,1,1,1}",2.0,8.0,"{{2,{5}},{3,{1}},{4,{5}},{5,{2,2,2,2}},{6,{5}}...",2
3,5_2,5,5.2,Y,5,5a_1,4,"[4, 8, 10, 2, 6]",5_2,[32],...,Y,Y,does not exist,"{{1,5,2,4},{3,9,4,8},{5,1,6,10},{7,3,8,2},{9,7...","{1,1,2,{2,1,-2}}","{1,1,2,{2,1,-2}}",1.0,8.0,"{{2,{7}},{3,{5,5}},{4,{3,21}},{5,{11,11}},{6,{...",1
4,6_1,6,6.1,Y,6,6a_3,8,"[4, 8, 12, 10, 2, 6]",6_1,[42],...,N,N,does not exist,does not exist,does not exist,does not exist,0.0,8.0,"{{2,{9}},{3,{7,7}},{4,{5,45}},{5,{31,31}},{6,{...",1
5,6_2,6,6.2,Y,7,6a_2,7,"[4, 8, 10, 12, 2, 6]",6_2,[312],...,N,N,does not exist,does not exist,does not exist,does not exist,1.0,8.0,"{{2,{11}},{3,{5,5}},{4,{11}},{5,{2,2,2,2}},{6,...",2
6,6_3,6,6.3,Y,8,6a_1,6,"[4, 8, 10, 2, 12, 6]",6_3,[2112],...,N,N,does not exist,does not exist,does not exist,does not exist,1.0,8.0,"{{2,{13}},{3,{7,7}},{4,{3,39}},{5,{4,4,4,4}},{...",2
7,xq,7,7.1,Y,9,7a_7,15,"[8, 10, 12, 14, 2, 4, 6]",7_1,[7],...,Y,Y,"{1,1,1,1,1,1,1}","{{1,9,2,8},{3,11,4,10},{5,13,6,12},{7,1,8,14},...","{1,1,1,1,1,1,1}","{1,1,1,1,1,1,1}",3.0,8.0,"{{2,{7}},{3,{1}},{4,{7}},{5,{1}},{6,{7}},{7,{2...",3
8,7_2,7,7.2,Y,10,7a_4,12,"[4, 10, 14, 12, 2, 8, 6]",7_2,[52],...,Y,Y,does not exist,"{{2,10,3,9},{4,14,5,13},{6,12,7,11},{8,2,9,1},...","{1,1,{3,2,-3},{2,1,-2},3}","{1,1,{3,2,-3},{2,1,-2},3}",1.0,8.0,"{{2,{11}},{3,{8,8}},{4,{5,55}},{5,{31,31}},{6,...",1
9,7_3,7,7.3,Y,11,7a_5,13,"[6, 10, 12, 14, 2, 4, 8]",7_3,[43],...,Y,Y,does not exist,"{{1,9,2,8},{3,11,4,10},{5,1,6,14},{7,13,8,12},...","{1,1,1,1,2,{2,1,-2}}","{1,1,1,1,2,{2,1,-2}}",2.0,8.0,"{{2,{13}},{3,{4,4}},{4,{13}},{5,{19,19}},{6,{8...",2
10,7_4,7,7.4,Y,12,7a_6,14,"[6, 10, 12, 14, 4, 2, 8]",7_4,[313],...,Y,Y,does not exist,"{{2,10,3,9},{4,12,5,11},{6,14,7,13},{8,4,9,3},...","{1,{3,2,-3},{3,2,1,-2,-3},{2,1,-2},3}","{1,{3,2,-3},{3,2,1,-2,-3},{2,1,-2},3}",2.0,8.0,"{{2,{15}},{3,{11,11}},{4,{7,105}},{5,{61,61}},...",2


In [11]:
three_colorable_knots = df[df["determinant"]%3==0]

In [12]:
three_colorable_gauss = three_colorable_knots[["name","gauss_notation"]]

In [13]:
three_colorable_gauss.to_csv("./data/name_gaussfromgauss_3colorable.csv",index=False)

In [18]:
five_colorable_braid = five_colorable_knots[["name","braid_notation"]]

In [20]:
five_colorable_braid.to_csv("name_braid_5colorable.csv",index=False)

# to-do 
- make a function that compares the gauss and braid DLNs
    - we are dealing with list of lists with [[this is the first coloring,this is a check of the first coloring],[ this is another coloring,checking another coloring with a different method]] 
    - first check to see if each individual one
    - get the gauss and braid DLN
    - switch every element to abs
    - if element in gauss code is in 

- write a function that determines whether a number is prime lol
- get a list of prime determinants of all knots
- get all fields for all gauss and braids

In [2]:
DLN_gauss = pd.read_csv("./data/name_DLNfromgauss_3colorable.csv")[1:]#. index_col = "name")
DLN_braid = pd.read_csv("./data/name_DLNfrombraid_3colorable.csv")[1:]

In [25]:
DLN_gauss_frac = []
knot_list = []

for knot in DLN_gauss["Dihedral Linking Number"]:#.values.tolist():
#     print(knot)
    strings = knot[2:-2].split("], [")
#     print(strings)
    for string in strings:
        if string == "'No Dihedral Linking Number Exists'":
            knot_list.append("No Dihedral Linking Number Exists")
        else:
            knot_list.append(abs(Fraction(string)))
    DLN_gauss_frac.append(knot_list)
    knot_list = []

# DLN_gauss_frac

In [26]:
DLN_braid_frac = []
knot_list = []

for knot in DLN_braid["Dihedral Linking Number"]:#.values.tolist():
#     print(knot)
    strings = knot[2:-2].split("], [")
#     print(strings)
    for string in strings:
        if string == "'No Dihedral Linking Number Exists'":
            knot_list.append("No Dihedral Linking Number Exists")
        else:
            knot_list.append(abs(Fraction(string)))
    DLN_braid_frac.append(knot_list)
    knot_list = []

#DLN_braid_frac

In [27]:
for i in range(len(DLN_braid_frac)): #you need to do if in and remove... :))))
    for j in range(len(DLN_braid_frac[i])):
        if DLN_braid_frac[i][j] in DLN_gauss_frac[i]:
            DLN_braid_frac[i][j] = "yes" #DLN_gauss_frac[i].pop()
        else:
            DLN_braid_frac[i][j] = "NOOoooooooOOOOoooo"

yes_list = DLN_braid_frac    

In [31]:
for y in yes_list:
    for z in y:
        if z == 'yes':
            continue
        else:
            print(y)